In [1]:
import json
import cv2
import time
import joblib
import numpy as np
from confluent_kafka import Producer, Consumer
from tensorflow.keras.models import load_model,model_from_json

C:\Users\Hassa\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
json_file=open(r'C:\Users\Hassa\emotion_model.json','r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights(r'C:\Users\Hassa\emotion_model.h5')

json_file=open(r'C:\Users\Hassa\Age_model.json','r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights(r'C:\Users\Hassa\Age_model.h5')

music_model = joblib.load('music_model3.pkl')

In [8]:
gender_dic = {0:'Male', 1:'Female'}
emotion_dic = {0:'Angry',1:'Disgusted',2:'Fearful',3:'Happy',4:'Neutral',5:'Sad',6:'Surprised'}

conf = {'bootstrap.servers': "localhost:9092"}
producer = Producer(conf)

topic = "face"

cap = cv2.VideoCapture(r"Downloads\3248275-uhd_3840_2160_25fps.mp4")

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    
    face_detector = cv2.CascadeClassifier(r'C:\Face Recognition Dataset\haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y+h, x:x+w]
        cropped_img = cv2.resize(roi_gray_frame, (128, 128))
        cropped_img = cropped_img.astype(np.float32) / 255.
        cropped_img = np.expand_dims(cropped_img, axis=0)
        cropped_img = np.expand_dims(cropped_img, axis=-1)

        model_prediction = model.predict(cropped_img)
        gender_index = round(model_prediction[0][0][0])
        age = round(model_prediction[1][0][0])
        predicted_gender = gender_dic[gender_index]
        
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,(48,48)),-1),0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        predicted_emotion = emotion_dic[maxindex]
        
        x = 1 if predicted_gender == 'Male' else 0
        y = 1 if predicted_gender == 'Female' else 0
        z = 1 if predicted_emotion == 'Happy' else 0
        a = 1 if predicted_emotion == 'Sad' else 0
        b = 1 if predicted_emotion == 'Neutral' else 0
        c = 1 if age <= 30 else 0
        d = 1 if age <= 40 else 0
        e = 1 if age <= 50 else 0
        f = 1 if age <= 60 else 0
        g = 1 if age <= 70 else 0
        h = 1 if age <= 80 else 0
        i = 1 if age <= 90 else 0
        features = [[x,y,z,a,b,c,d,e,f,g,h,i]]
        music_prediction = music_model.predict(features)
        music_prediction_list = music_prediction.tolist()    

        message = {
            "gender": predicted_gender,
            "age": age,
            "emotion": predicted_emotion,
            "predicted_music": music_prediction_list
        }

        json_message = json.dumps(message)

        producer.produce(topic, key=None, value=json_message)

        print("Message sent to Kafka:", json_message)

    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
    time.sleep(40)

producer.flush()
producer.close()
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 31ms/step
Message sent to Kafka: {"gender": "Female", "age": 22, "emotion": "Happy", "predicted_music": ["Bobby Darin - Dream Lover"]}
1/1 [==============================] - ETA: 0s

C:\Users\Hassa\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 31ms/step
Message sent to Kafka: {"gender": "Female", "age": 26, "emotion": "Happy", "predicted_music": ["Bobby Darin - Dream Lover"]}
1/1 [==============================] - 0s 25ms/step
Message sent to Kafka: {"gender": "Male", "age": 28, "emotion": "Happy", "predicted_music": ["Avicii - Wake Me Up"]}


C:\Users\Hassa\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\Hassa\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


KeyboardInterrupt: 